## 두 장소 간 주소 변환 후 경로 탐색 기능

** 장소 검색 (네이버 Local API)

search_place(place_name)
→ 네이버 검색 API를 이용하여 입력된 장소의 주소를 가져옴.


** 좌표 변환 (네이버 Geocoding API)

get_place_details(place_name)
→ 검색된 주소를 기반으로 좌표(위도, 경도) 변환 수행.
→ 검색 실패 시, 네이버 지도 웹 검색 URL 제공.


** 네이버 지도 길찾기 링크 생성

generate_naver_map_link(start_coords, end_coords, start_label, end_label)
→ 출발지 & 도착지 정보를 기반으로 네이버 지도 길찾기 URL을 생성.


@@@사용자 입력을 받아 실행@@@
start_name과 end_name을 입력받아 좌표 변환 후 네이버 지도 웹 페이지 열기.

** 코드 실행 흐름름

사용자로부터 출발지와 도착지 입력받음.
네이버 검색 API로 장소의 주소를 먼저 검색 (검색 실패 시 원래 이름 사용).
네이버 Geocoding API를 통해 주소를 좌표(위도, 경도)로 변환.
출발지와 도착지의 좌표를 네이버 지도 길찾기 URL 형식으로 변환.
완성된 네이버 지도 길찾기 링크를 출력하고 자동으로 웹 브라우저에서 실행.

In [3]:
import os
import requests
import webbrowser
from urllib.parse import quote
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv("C:/Users/SAMSUNG/Desktop/2025 GDGoC winter 프로젝트/API Key.env", override=True)

# 네이버 API 인증 정보
NAVER_MAP_CLIENT_ID = os.getenv("NAVER_MAP_CLIENT_ID")
NAVER_MAP_CLIENT_SECRET = os.getenv("NAVER_MAP_CLIENT_SECRET")
NAVER_LOCAL_API_CLIENT_ID = os.getenv("NAVER_Local_API_CLIENT_ID")
NAVER_LOCAL_API_CLIENT_SECRET = os.getenv("NAVER_Local_API_CLIENT_SECRET")

def search_place(place_name):
    """네이버 검색 API(Local API)를 사용하여 장소의 주소 가져오기"""
    url = "https://openapi.naver.com/v1/search/local.json"
    headers = {
        "X-Naver-Client-Id": NAVER_LOCAL_API_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_LOCAL_API_CLIENT_SECRET,
    }
    params = {"query": place_name, "display": 1, "sort": "random"}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        if data["items"]:
            return data["items"][0]["address"]
        else:
            return None
    else:
        return None

def get_place_details(place_name):
    """네이버 Geocoding API를 사용하여 장소 좌표 및 주소 가져오기"""
    search_address = search_place(place_name)  # 먼저 검색 API로 주소 찾기
    query = search_address if search_address else place_name  # 주소가 있으면 주소로, 없으면 원래 이름으로 검색

    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": NAVER_MAP_CLIENT_ID,
        "X-NCP-APIGW-API-KEY": NAVER_MAP_CLIENT_SECRET,
    }
    params = {"query": query}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()

    if data.get("addresses"):
        location = data["addresses"][0]
        lat, lng = float(location["y"]), float(location["x"])
        address = location.get("roadAddress") or location.get("jibunAddress") or search_address or place_name
        print(f"✅ '{place_name}' → 위도 {lat}, 경도 {lng}, 주소: {address}")
        return (lng, lat), address  # (경도, 위도), 주소
    else:
        print(f"❌ '{place_name}'의 정보를 찾을 수 없습니다.")
        print(f"🌍 네이버 지도에서 직접 검색해보세요: https://map.naver.com/v5/search/{quote(place_name)}")
        return None, place_name

def generate_naver_map_link(start_coords, end_coords, start_label, end_label):
    """네이버 지도 길찾기 링크 생성"""
    start_lng, start_lat = start_coords
    end_lng, end_lat = end_coords
    full_url = (
        f"https://map.naver.com/p/directions/"
        f"{start_lng},{start_lat},{quote(start_label)},PLACE_POI/"
        f"{end_lng},{end_lat},{quote(end_label)},PLACE_POI/-/car"
    )
    return full_url

if __name__ == "__main__":
    print("📍 네이버 지도 길찾기 프로그램")
    
    start_name = input("출발지를 입력하세요: ").strip()
    start_coords, start_label = get_place_details(start_name)
    
    end_name = input("도착지를 입력하세요: ").strip()
    end_coords, end_label = get_place_details(end_name)
    
    if start_coords and end_coords:
        naver_map_link = generate_naver_map_link(start_coords, end_coords, start_label, end_label)
        print(f"🌍 네이버 지도 경로 링크: {naver_map_link}")
        webbrowser.open(naver_map_link)
    else:
        print("❌ 출발지 또는 도착지 좌표를 가져올 수 없습니다.")


📍 네이버 지도 길찾기 프로그램
✅ '남산타워' → 위도 37.5489463, 경도 126.9894131, 주소: 서울특별시 용산구 남산공원길 105 YTN서울타워
✅ '롯데월드' → 위도 37.5112949, 경도 127.0979688, 주소: 서울특별시 송파구 송파대로 521 잠실자전거수리센터
🌍 네이버 지도 경로 링크: https://map.naver.com/p/directions/126.9894131,37.5489463,%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%9A%A9%EC%82%B0%EA%B5%AC%20%EB%82%A8%EC%82%B0%EA%B3%B5%EC%9B%90%EA%B8%B8%20105%20YTN%EC%84%9C%EC%9A%B8%ED%83%80%EC%9B%8C,PLACE_POI/127.0979688,37.5112949,%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%86%A1%ED%8C%8C%EA%B5%AC%20%EC%86%A1%ED%8C%8C%EB%8C%80%EB%A1%9C%20521%20%EC%9E%A0%EC%8B%A4%EC%9E%90%EC%A0%84%EA%B1%B0%EC%88%98%EB%A6%AC%EC%84%BC%ED%84%B0,PLACE_POI/-/car


✅ 네이버 검색 API + Geocoding API를 결합하여 보다 정확한 검색 결과 제공
✅ 주소 검색이 실패할 경우 네이버 지도 검색 링크 제공
✅ 자동으로 네이버 지도에서 길찾기 링크 생성 및 실행

## 장소 정보 제공 -> Naver Map, Google 검색 web URL

** 네이버 지도에서 장소 검색 (search_naver_map)

사용자가 입력한 장소명을 네이버 지도에서 검색할 수 있도록 URL을 생성합니다.
webbrowser.open(url)을 사용하여 자동으로 네이버 지도 검색 결과 페이지를 엽니다.
예: https://map.naver.com/v5/search/남산타워


** 구글 검색을 활용한 주소 찾기 (search_google_for_address)

장소명을 그대로 구글 검색 URL에 적용하여 검색합니다.
webbrowser.open(url)을 사용하여 자동으로 구글 검색 결과 페이지를 엽니다.
예: https://www.google.com/search?q=남산타워


메인 실행부 (__main__)

사용자에게 검색할 장소명을 입력받음 (input).
입력된 장소명을 search_naver_map(place_name)을 호출하여 네이버 지도에서 검색.
이어서 search_google_for_address(place_name)을 호출하여 구글에서 검색.

** 사용자가 장소명을 입력
네이버 지도에서 검색 → search_naver_map(place_name) 실행
구글 검색에서 검색 → search_google_for_address(place_name) 실행
웹 브라우저에서 각각의 검색 결과 페이지가 열림

💡 API 없이 웹 브라우저에서 자동 검색을 수행하는 방식

In [6]:
import os
from dotenv import load_dotenv  
import requests

# 🔍 네이버 검색 URL을 이용한 검색
def search_naver_map(place_name):
    """네이버 지도에서 직접 검색 URL 생성 및 실행"""
    encoded_place = quote(place_name)  # URL 인코딩 (한글 포함)
    search_url = f"https://map.naver.com/v5/search/{encoded_place}"
    print(f"🌍 네이버 지도 검색 URL: {search_url}")
    webbrowser.open(search_url)

# 🔍 구글 검색을 활용한 주소 가져오기 (네이버 API 없이 검색 기반)
def search_google_for_address(place_name):
    """구글 검색 엔진을 활용하여 실제 정보를 가져오기"""
    query = f"{place_name}"
    search_url = f"https://www.google.com/search?q={quote(query)}"

    print(f"🔍 구글에서 검색 중: {search_url}")
    webbrowser.open(search_url)  # 웹 브라우저에서 검색 실행

if __name__ == "__main__":
    place_name = input("장소명을 입력하세요: ").strip()

    # 1️⃣ 네이버 지도에서 직접 검색
    search_naver_map(place_name)

    # 2️⃣ 구글 검색을 활용하여 검색
    search_google_for_address(place_name)


🌍 네이버 지도 검색 URL: https://map.naver.com/v5/search/%EC%8B%9C%EC%86%8C%EC%B9%B4%EC%9D%B4%EB%A7%89
🔍 구글에서 검색 중: https://www.google.com/search?q=%EC%8B%9C%EC%86%8C%EC%B9%B4%EC%9D%B4%EB%A7%89
